## 2024 TBA Predictions

Match data is fetched from TBA by running fetchMatches.py.  Run this first before running this notebook.

`python fetchMatches.py`


In [1]:
from __future__ import print_function
import pickle
#import time
import sys
from collections import Counter
from tqdm import tqdm

sys.path.append('..')
import swagger_client as v3client
from swagger_client.rest import ApiException

# If you fetch_matches best to set reset=True or you may miss some events.
fetch_matches = False
reset = False
year = 2024

if fetch_matches:
    from fetchMatches import fetch_all_matches
    # This will save to matches_{year}.pkl
    fetch_all_matches(year, reset=reset)


In [2]:
# load all the matches

filename = f'matches_{year}.pkl'
matches = []
with open(filename, 'rb') as f:
    matches = pickle.load(f)

In [3]:
# [m for m in matches['events'] if m.address and 'Spokane' in m.address]
#[m for m in matches['events'] if 'pnc' in m.key]
pnw_district = [m.key for m in matches['events'] ] # if m.district and m.district.abbreviation=='pnw']
pnw_district

['2024alhu',
 '2024arc',
 '2024arli',
 '2024ausc',
 '2024azdd',
 '2024azgl',
 '2024azva',
 '2024bcvi',
 '2024brbr',
 '2024caav',
 '2024cabe',
 '2024cada',
 '2024cafr',
 '2024cala',
 '2024camb',
 '2024caoc',
 '2024caph',
 '2024casd',
 '2024casf',
 '2024casj',
 '2024cave',
 '2024chcmp',
 '2024cmptx',
 '2024cocs',
 '2024code',
 '2024cthar',
 '2024ctwat',
 '2024cur',
 '2024dal',
 '2024flor',
 '2024flta',
 '2024flwp',
 '2024gaalb',
 '2024gacar',
 '2024gacmp',
 '2024gadal',
 '2024gagwi',
 '2024gal',
 '2024hiho',
 '2024hop',
 '2024iacf',
 '2024idbo',
 '2024ilch',
 '2024ilpe',
 '2024incmp',
 '2024incol',
 '2024ineva',
 '2024inmis',
 '2024inpla',
 '2024iscmp',
 '2024isde1',
 '2024isde2',
 '2024isde3',
 '2024isde4',
 '2024johnson',
 '2024ksla',
 '2024lake',
 '2024mabos',
 '2024mabri',
 '2024marea',
 '2024mawne',
 '2024mawor',
 '2024mdowi',
 '2024mdsev',
 '2024melew',
 '2024miann',
 '2024mibat',
 '2024mibel',
 '2024miber',
 '2024mibkn',
 '2024mibro',
 '2024micmp',
 '2024midet',
 '2024midtr',
 '20

Filter the matches to completed matches

In [4]:
non_empty = [k for k in matches['matches'].keys() if len(matches['matches'][k])>0]
data = [m for k in matches['matches'] for m in matches['matches'][k]]
data = [m for m in data if m.winning_alliance!='' and m.score_breakdown is not None]
print(f'Found {len(data)} matches')

pnw_teams = set()
for m in [m for m in data if m.event_key in pnw_district]:
    for t in m.alliances.red.team_keys:
        pnw_teams.add(t)
    for t in m.alliances.blue.team_keys:
        pnw_teams.add(t)
    
pnw_teams = list(sorted(pnw_teams))
print(f'PNW Teams: {pnw_teams}')
#red = [x for m in data for x in m.alliances.red.team_keys]
#blue = [x for m in data for x in m.alliances.blue.team_keys]
#from collections import Counter
#Counter(red+blue)

Found 6257 matches
PNW Teams: ['frc1', 'frc100', 'frc1014', 'frc1018', 'frc1023', 'frc1024', 'frc1025', 'frc103', 'frc1038', 'frc1051', 'frc1056', 'frc1058', 'frc107', 'frc1071', 'frc1072', 'frc1073', 'frc1076', 'frc108', 'frc1086', 'frc1089', 'frc1091', 'frc1099', 'frc11', 'frc1100', 'frc1102', 'frc111', 'frc1114', 'frc1119', 'frc112', 'frc1126', 'frc1138', 'frc114', 'frc1143', 'frc1148', 'frc115', 'frc1153', 'frc1155', 'frc1156', 'frc1159', 'frc1160', 'frc1164', 'frc1165', 'frc1168', 'frc117', 'frc118', 'frc1188', 'frc1189', 'frc1197', 'frc120', 'frc1208', 'frc1209', 'frc1218', 'frc122', 'frc1225', 'frc123', 'frc1241', 'frc1247', 'frc125', 'frc1250', 'frc1255', 'frc1257', 'frc1259', 'frc126', 'frc1262', 'frc1277', 'frc1279', 'frc1285', 'frc1287', 'frc1293', 'frc1294', 'frc1296', 'frc1305', 'frc1306', 'frc1307', 'frc1308', 'frc131', 'frc1310', 'frc1318', 'frc1319', 'frc1322', 'frc1325', 'frc133', 'frc1339', 'frc135', 'frc1350', 'frc1351', 'frc1359', 'frc1360', 'frc138', 'frc1388', 'fr

In [5]:
pnw_matches = [m for m in data if m.event_key in pnw_district]
pnw_lookup = dict((k,i) for (i,k) in enumerate(pnw_teams))

In [6]:
# We'll only train based on qualifier matches
qualifiers = [x for x in data if x.comp_level=='qm'] 

Create aggregate team statistics for all teams, and a separate set for PNW teams

In [7]:
pnw_matches[0]
def get_rows(m):
    yield m.alliances.red.team_keys, m.score_breakdown['red']['totalPoints']
    yield m.alliances.blue.team_keys, m.score_breakdown['blue']['totalPoints']
list(get_rows(pnw_matches[0]))

[(['frc16', 'frc3937', 'frc6586'], 84),
 (['frc4336', 'frc9429', 'frc2357'], 60)]

In [8]:
from scipy.sparse import csr_array
from scipy.sparse.linalg import spsolve, norm
import numpy as np

data = []
row = []
col = []
b = []
ctr = 0
for m in map(get_rows, pnw_matches):
    for r in m:
        for t in r[0]:
            row.append(len(b))
            col.append(pnw_lookup[t])
            data.append(1)
        b.append(r[1])
b = np.array(b)            
A = csr_array((data, (row, col)), shape=(len(b), len(pnw_lookup)))
print(A.shape, b.shape )
#x = spsolve(A, b)
#x
from scipy.sparse.linalg import lsqr
from scipy.linalg import lstsq

# Thanks ChatGPT!
x, residuals, rank, s = lstsq(A.todense(), b)

RSS = residuals.sum()
Rinv = np.linalg.inv(np.triu(s))

sigmas = np.sqrt(RSS / (len(b) - len(x)) * np.diag(Rinv))

#return_values = lsqr(A, b, calc_var=True)
#result = return_values
#print(result)
#x = return_values[0]
#var = return_values[-1]
#print(var)

opr = [(t,x[i],sigmas[i]) for i,t in enumerate(pnw_teams)]
for t,opr,sigma in sorted(opr, key=lambda x: x[1], reverse=True):
    print(t,opr,sigma)

#print((A@x).shape,b.shape)
print(A@x-b)
err = np.mean(A@x-b)
print(err)
opr_lookup = dict([(t,(x[i],sigmas[i])) for i,t in enumerate(pnw_teams)])


(12514, 2713) (12514,)
frc599 104162094711498.16 0.0
frc702 104162094711484.08 0.0
frc7230 104162094711441.88 0.0
frc1160 56503754631956.19 0.0
frc7137 56503754631938.81 0.0
frc9564 47658340079550.84 0.0
frc5669 47658340079520.18 0.0
frc2194 5551485372108.674 0.0
frc1259 5551485372106.908 0.0
frc4531 5551485372099.55 0.0
frc3596 5551485372064.729 0.0
frc8847 5551485372057.016 0.0
frc2202 5551485372053.497 0.0
frc2220 5551485372032.1045 0.0
frc8744 5551485372030.451 0.0
frc2506 5551485372028.011 0.0
frc537 5551485371994.892 0.0
frc2667 75.51775150168406 0.0
frc4564 60.56062238158628 0.0
frc2056 60.3919964154353 0.0
frc5902 59.62795431763564 0.0
frc5148 59.49947681396189 0.0
frc254 57.375389636525306 0.0
frc4079 55.33130318963094 0.0
frc1678 54.328426197050376 0.0
frc1690 52.205925265371924 0.0
frc2481 51.016550648595384 0.0
frc5940 50.275139513159445 0.0
frc3543 50.23209229877663 0.0
frc4999 49.89403049012399 0.0
frc4613 49.8167901066103 0.0
frc4546 49.05866334240236 0.0
frc6329 48.3034

frc6300 20.778864434529904 0.0
frc2926 20.77328354865319 0.0
frc151 20.757841443743303 0.0
frc386 20.73796212074216 0.0
frc5809 20.726321906234595 0.0
frc1577 20.725737617738364 0.0
frc5684 20.69994583411284 0.0
frc8608 20.682482600348987 0.0
frc3984 20.67706299502387 0.0
frc3102 20.676975177064232 0.0
frc558 20.67124747642948 0.0
frc5740 20.65290869689081 0.0
frc3954 20.59847605999878 0.0
frc5561 20.588080702807893 0.0
frc9576 20.582348800825248 0.0
frc4663 20.579608320392758 0.0
frc8898 20.568971962143756 0.0
frc1768 20.540880960241488 0.0
frc6418 20.529931445005758 0.0
frc5914 20.521663045401368 0.0
frc5291 20.521570460773937 0.0
frc8576 20.509514097186763 0.0
frc957 20.497442625463037 0.0
frc5924 20.49304193682869 0.0
frc2010 20.473400238715428 0.0
frc3939 20.47144559131182 0.0
frc177 20.44943531028974 0.0
frc7154 20.431326942146455 0.0
frc2883 20.413132061930316 0.0
frc5166 20.41200707450534 0.0
frc3316 20.388222769721178 0.0
frc6962 20.365567029210798 0.0
frc3142 20.3650064070518

frc1561 14.788760640713715 0.0
frc3637 14.767347501059797 0.0
frc9674 14.75525360995252 0.0
frc8780 14.746201485415039 0.0
frc2846 14.742846422110725 0.0
frc2423 14.733596077511955 0.0
frc4511 14.720827884590433 0.0
frc9594 14.713022587609068 0.0
frc5507 14.707136680764656 0.0
frc5253 14.690344055802813 0.0
frc1014 14.676392529683824 0.0
frc188 14.644236480508573 0.0
frc3620 14.644096149274938 0.0
frc4723 14.64207250298821 0.0
frc3418 14.639462155961894 0.0
frc9607 14.636199801184087 0.0
frc5086 14.63148083415405 0.0
frc3926 14.614850371320436 0.0
frc2813 14.61321135879469 0.0
frc117 14.607196532660469 0.0
frc930 14.586061784627281 0.0
frc1592 14.556263246889747 0.0
frc4338 14.549886964502166 0.0
frc7656 14.546308591764745 0.0
frc4809 14.543054196212879 0.0
frc8839 14.54278060990914 0.0
frc5141 14.438033506010157 0.0
frc1533 14.36675125625996 0.0
frc5535 14.338285484264812 0.0
frc587 14.30945613641332 0.0
frc9433 14.306419065447884 0.0
frc2898 14.268829196691541 0.0
frc6344 14.25457296

frc6419 10.759446684678991 0.0
frc7287 10.755854436515474 0.0
frc5503 10.742899879599218 0.0
frc5724 10.7422145962794 0.0
frc3826 10.725339199882004 0.0
frc3298 10.718211345871985 0.0
frc7465 10.706047492202641 0.0
frc3958 10.705118841744547 0.0
frc5417 10.702914914129419 0.0
frc334 10.699739841841112 0.0
frc8051 10.695508863427678 0.0
frc9486 10.686005344406906 0.0
frc2064 10.670974885647276 0.0
frc8096 10.656145030991027 0.0
frc2972 10.653449778314458 0.0
frc3568 10.65038808044034 0.0
frc9492 10.643807068775724 0.0
frc2523 10.637066421444807 0.0
frc8119 10.613250690146911 0.0
frc1997 10.609589149460374 0.0
frc2560 10.602042525191411 0.0
frc9100 10.558483083351293 0.0
frc5183 10.557404962511683 0.0
frc9056 10.551129930718952 0.0
frc5197 10.534385736042905 0.0
frc4026 10.516581912729148 0.0
frc6512 10.506758653848825 0.0
frc6420 10.50648597212057 0.0
frc8089 10.499047433531725 0.0
frc4973 10.490239978931347 0.0
frc1287 10.4817914164926 0.0
frc5113 10.461801361242218 0.0
frc9725 10.4553

frc5516 7.195518519250024 0.0
frc3487 7.195019187592179 0.0
frc7072 7.194359262633178 0.0
frc2927 7.189859005273322 0.0
frc5697 7.185000823683712 0.0
frc8261 7.17089430802516 0.0
frc9307 7.159010093139525 0.0
frc9006 7.138090049483965 0.0
frc5189 7.137480371551751 0.0
frc5407 7.135554327154032 0.0
frc253 7.123164338645106 0.0
frc5150 7.113032165454436 0.0
frc2224 7.096439483708757 0.0
frc5929 7.088480525848196 0.0
frc5295 7.088409463409337 0.0
frc6333 7.083279943083507 0.0
frc6171 7.072664105053133 0.0
frc7417 7.063807537793352 0.0
frc5000 7.054569343455024 0.0
frc5834 7.051538926755238 0.0
frc7477 7.009794084411829 0.0
frc5103 7.007590446106441 0.0
frc5248 7.003002873924617 0.0
frc9301 6.9952483611149106 0.0
frc8060 6.9945873705293575 0.0
frc9524 6.9926881934178775 0.0
frc4827 6.981581925643387 0.0
frc904 6.9710480619947095 0.0
frc6961 6.966987432811419 0.0
frc9617 6.958040360473078 0.0
frc9693 6.952007935829756 0.0
frc2903 6.939086126163611 0.0
frc8803 6.937569591742115 0.0
frc9155 6

frc5012 4.787586880428583 0.0
frc8584 4.77969324842781 0.0
frc4918 4.7722256567794865 0.0
frc4348 4.771073664216102 0.0
frc6988 4.762872987655809 0.0
frc5135 4.7559463139232205 0.0
frc3288 4.747446139144312 0.0
frc2950 4.739321885057058 0.0
frc744 4.729458701704516 0.0
frc781 4.725796684333959 0.0
frc9156 4.719793226929748 0.0
frc8175 4.715509075937987 0.0
frc9203 4.706516022499686 0.0
frc9303 4.7039354318152595 0.0
frc5954 4.69962503682216 0.0
frc8708 4.698293939248332 0.0
frc7713 4.698142426850994 0.0
frc4229 4.690160151635212 0.0
frc2190 4.688464487469122 0.0
frc5430 4.682650995114512 0.0
frc8369 4.680181725258073 0.0
frc7418 4.678189244803575 0.0
frc6096 4.671455500086189 0.0
frc3966 4.660320068861922 0.0
frc8863 4.651875434122896 0.0
frc5071 4.649696608145068 0.0
frc1539 4.648559572548009 0.0
frc7725 4.647213187273761 0.0
frc7900 4.64336837828353 0.0
frc6411 4.617232125665678 0.0
frc404 4.612986874221655 0.0
frc5205 4.603729447044452 0.0
frc8704 4.597546701507944 0.0
frc9668 4.591

Next steps: for a given event we want to decide how to prioritize alliance choices.  Suppose we have all the data to date, as well as all the qualifier data for the event. Who should we choose as partners?

In [9]:

def predict(red,blue):
    mu = []
    sigma = []
    for r in red:
        mu.append(opr_lookup[r][0])
        sigma.append(opr_lookup[r][1])
    for b in blue:
        mu.append(-opr_lookup[b][0])
        sigma.append(opr_lookup[b][1])
    mu = sum(mu)
    sigma = np.linalg.norm(sigma)
    return(mu,sigma)

#predict(['frc2910'],['frc492'])
predict(['frc78'],['frc88'])


(15.869023663314906, 0.0)

In [10]:
## Brackets

alliances =  {
 'A1':   [7461, 2910,5827],
 'A2':  [488, 360, 4450],
 'A3':  [4911, 2412, 4512],
 'A4':  [5937, 1983, 3070],
 'A5':   [2976, 1899, 9023],
 'A6':   [492, 4682, 3681],
 'A7':   [1294, 948, 8248],
 'A8':   [9036, 949, 2928]
}

for k in alliances:
    alliances[k] = list(map(lambda x: f'frc{x}', alliances[k]))

bracket = {
    1: ['A1', 'A8'],
    2: ['A4', 'A5'],
    3: ['A2', 'A7'],
    4: ['A3', 'A6'],
    5: ['L1', 'L2'],
    6: ['L3', 'L4'],
    7: ['W1', 'W2'],
    8: ['W3', 'W4'],
    9: ['L7', 'W6'],
    10: ['W5', 'L8'],
    11: ['W7', 'W8'],
    12: ['W10', 'W9'],
    13: ['L11', 'W12'],
    14: ['W11', 'W13'],
    15: ['W14', 'L14'],
    16: ['W15', 'L15']
}

density = {i:Counter() for i in range(1,len(bracket)+1)}
        
def runMatch(matchNumber):
    red_id,blue_id = bracket[matchNumber]
    
    red = alliances[red_id]
    blue =alliances[blue_id]
    density[matchNumber][str(red)]+=1
    density[matchNumber][str(blue)]+=1
    #density[matchNumber][red_id]+=1
    #density[matchNumber][blue_id]+=1
    
    # mu and sigma are the expected advantage for red
    mu,sigma = predict(red,blue)
    r = np.random.normal(mu,sigma)
    #print(red,blue,mu,sigma,r)
    
    if r>0:        
        winner = red
        loser = blue
    else:
        winner = blue
        loser = red
    alliances[f'W{matchNumber}'] = winner
    alliances[f'L{matchNumber}'] = loser
    #print(f'{winner} beats {loser} by {abs(r)} in match {matchNumber}')

    
def pMatch(matchNumber):
    red_id,blue_id = bracket[matchNumber]
    
    red = alliances[red_id]
    blue =alliances[blue_id]
    density[matchNumber][str(red)]+=1
    density[matchNumber][str(blue)]+=1
    #density[matchNumber][red_id]+=1
    #density[matchNumber][blue_id]+=1
    
    # mu and sigma are the expected advantage for red
    return predict(red,blue)

import scipy.stats as stats

def pRed(matchNumber):
    mu,sigma = pMatch(matchNumber)
    return 1.0-stats.norm.cdf(0, loc=mu, scale=sigma)
    
    
def runBracket():
    for i in range(1,17):
        runMatch(i)        
    wins = Counter()
    for i in range(14,17):
        w = alliances[f'W{i}']
        wins[str(w)]+=1
    return sorted(wins, reverse=True, key=lambda x: wins[x])[0], (alliances['A6'] in [alliances['W11'],alliances['W13']])

overall = Counter()
inFinalCtr = 0
for b in tqdm(range(1000)):
    (w, inFinal) = runBracket()
    overall[w] += 1
    inFinalCtr += 1 if inFinal else 0
        
for k in sorted(overall, key=lambda x: overall[x], reverse=True):
    print(k, overall[k])

print(f'inFinal: {inFinalCtr}')

for k in sorted(density):
    print(k, density[k])

  0%|                                                                                         | 0/1000 [00:00<?, ?it/s]


KeyError: 'frc7461'

In [ ]:
import matplotlib.pyplot as plt
import matplotlib
import json
import seaborn as sns
colors = sns.color_palette(n_colors=8) # ('Dark2', n_colors=8)
cmap = matplotlib.colors.ListedColormap(colors)
plt.rcParams["figure.figsize"] = (20, 20)

allianceLookup = dict([(str(alliances[k]),k) for k in alliances if k[0]=='A'] )
alliance_list = list(allianceLookup.values())


def makelabel(s):
    #l = json.loads(s.replace('\'','"'))
    #return ','.join(map(lambda x: x[3:], l))
    return allianceLookup[s]
    
fig,axs= plt.subplots(len(density)//4, 4)
for a,i in enumerate(density):
    ax = axs[a//4,a%4]
    k = list(sorted(density[i], reverse=True, key=lambda d: density[i][d]))
    v = [density[i][d] for d in k]
    labels=list(map(makelabel, k))
    color_indices = [alliance_list.index(a) for a in labels]
    colors = [cmap(index) for index in color_indices]
    ax.pie(v, labels=labels, colors=colors, explode=[0.05]*len(labels)) #wedgeprops = {'linewidth': 3, 'linecolor':'white'})
    ax.set_title(f'Match {i}')

In [ ]:
# swap alliance 1 and 6 third choice

alliances =  {
 'A1':   [7461, 2910,3681],
 'A2':  [488, 360, 4450],
 'A3':  [4911, 2412, 4512],
 'A4':  [5937, 1983, 3070],
 'A5':   [2976, 1899, 9023],
 'A6':   [492, 4682, 5827],
 'A7':   [1294, 948, 8248],
 'A8':   [9036, 949, 2928]
}

for k in alliances:
    alliances[k] = list(map(lambda x: f'frc{x}', alliances[k]))


overall = Counter()
for b in tqdm(range(1000)):
    w = runBracket()
    overall[w] += 1
        
for k in sorted(overall, key=lambda x: overall[x], reverse=True):
    print(k, overall[k])

In [11]:
yakima_teams = set()

for m in [m for m in pnw_matches if m.event_key == '2024wayak']:
    for t in m.alliances.red.team_keys:
        yakima_teams.add(t)
    for t in m.alliances.blue.team_keys:
        yakima_teams.add(t)
yakima_teams

{'frc1318',
 'frc1595',
 'frc2147',
 'frc2811',
 'frc2926',
 'frc3574',
 'frc3712',
 'frc3876',
 'frc4060',
 'frc4061',
 'frc4089',
 'frc4104',
 'frc4125',
 'frc4131',
 'frc4513',
 'frc4692',
 'frc492',
 'frc4980',
 'frc5468',
 'frc5920',
 'frc6076',
 'frc6343',
 'frc6443',
 'frc6831',
 'frc8532',
 'frc9438',
 'frc9613',
 'frc9680'}

In [12]:
for t in sorted(yakima_teams, key = lambda t: opr_lookup[t][0], reverse=True):
    print(t,opr_lookup[t][0])

frc2811 34.52095665037639
frc492 27.909339806064978
frc4125 25.129392832517677
frc1318 23.31152430921795
frc4980 22.562279237434282
frc2147 22.316118869930524
frc4089 22.181578867137453
frc6443 20.902075342834053
frc2926 20.77328354865319
frc4513 19.652233500033635
frc3574 19.191941767931
frc1595 17.23186032846571
frc5468 16.12716082856064
frc4131 15.937636099755784
frc6343 15.60684999078519
frc8532 15.088183686137253
frc3876 14.150625813752434
frc6831 13.21081310510642
frc3712 12.264113202691135
frc4060 11.689022194594193
frc9680 11.513214167207492
frc5920 9.825559191405828
frc4061 8.063701253384398
frc6076 7.400586234405652
frc4104 7.225443897768866
frc9613 6.376221399754334
frc4692 5.375310691073549
frc9438 4.321795977652119


In [ ]:
opr_lookup